In [1]:
import os
import pandas as pd

In [2]:
# params
directory = '../sleuth/'

# sequences:
analysis = next(os.walk(directory))[1]

In [26]:
def sleuth_analysis(directory, genovar):
    """
    A function to write the differential_expression_analyzer batch command.
    """
    heart = 'Rscript diff_exp_analyzer.R -d {0} --genovar {1}'.format(directory, genovar)
    return heart

def walk_sleuth_directories(directory):
    """
    Given a directory, walk through it,
    find all the rna-seq repository folders
    and generate kallisto commands
    """
    sleuth = ''
    #directory contains all the projects, walk through it:
    current, dirs, files = next(os.walk(directory))
    for d in dirs:
        # genovar always begins with a z:
        genovar = 'z' + d[-1:]
        message = '# Sleuth analysis command for {0}\n'.format(d)
        command = sleuth_analysis(d, genovar) +'\n'
        sleuth += message
        sleuth += command
    return sleuth

In [27]:
with open('../sleuth/sleuth_commands.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    f.write('# Bash commands for diff. expression analysis using Sleuth.\n')
    sleuth_command = walk_sleuth_directories(directory)
    f.write(sleuth_command)
#     print(sleuth_command)

In [15]:
current, dirs, files = next(os.walk(directory))

In [16]:
dirs

['WT_A', 'WT_B', 'WT_C', 'WT_D', 'WT_E', 'WT_F', 'WT_wt']